In [1]:
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np

In [3]:
data = pd.read_csv('./data/news_data.csv', encoding='ISO-8859-1')
df = data.copy()
df.sample(5)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
3418,2013-10-16,1,"France finds Church of Scientology guilty of ""...",Glenn Greenwald is leaving The Guardian to lau...,Divers have recovered what may be a 570kg chun...,Brazilian judge rules in favour of farmers aga...,Jihadi Work Accident: Two Al-Shabaab Suicide B...,Transparency International calls on German Pre...,"End of Swiss bank secrecy, black money informa...","Iraq War death toll near 500K, new study estim...",...,"Thai Police Are Shocked, Shocked After Rihanna...","Iraq war claimed half a million lives, peer-re...",93 dead as 7.2 magnitude quake hits Philippine...,Malala Yousafzai to get honorary Canadian citi...,GoDaddy Pulls Lavabit's Security Creds Because...,Dutch diplomat assaulted in Russia,The number of cockroach farms in China is growing,"""I just want to have a future."" Anger in Franc...",Fukushima Industries said Tuesday it was rethi...,Lavabit comes back online for 96 hours to allo...
3407,2013-10-01,1,Ex-Microsoft privacy chief: I don't trust Micr...,US leaker Edward Snowden among 3 finalists for...,A CEO who resisted NSA spying is out of prison...,"Age of Unreason: The governments of Britain, C...",German writer and NSA-critic is not allowed to...,"""A superpower has paralyzed itself. The unthin...",Korean minister blasts Japan for covering up F...,"Snowden to EU: Whistleblowers need protection,...",...,"NSA spied on Martin Luther King, Muhammad Ali,...","Venezuela kicks out three U.S. diplomats, alle...",Saudi Men Buying Syrian Refugee Children as Br...,The Indian lawyer for two of the four Delhi ga...,NSA May Not Be Collecting Your Location Data F...,Qatar World Cup 2022: 70 Nepalese workers die ...,War on illegal drugs failing: Illegal drugs ar...,Putin nominated for Nobel Peace Prize,"Obama, after meeting Israel PM, says 'no optio...","At U.N., Syria compares rebel violence to 9/11..."
3938,2015-11-09,0,Catalan parliament votes to secede from Spain ...,TPP deal worst thing that Harper government ha...,Turkish Atheism Association starts petition ca...,World Anti-Doping Agency says Russia 'sabotage...,Thousands of unopened letters dating to the 17...,A new Taliban breakaway group claims support f...,Myanmar elections: ruling party concedes defea...,World Bank warns climate change could add 100 ...,...,Newly declassified documents reveal just how e...,Paralympian 'dragged self' from Qatar Airways ...,Chinese spies try to hack Australian submarine...,A dam holding back waste water from an iron or...,Germany's capacity to take in refugees is limi...,Warming set to breach 1C threshold,Canadian Doctor At Sunnybrook In Toronto First...,"Israel, US to initiate huge arms deal",Australia Deploys Sheepdogs to Save a Penguin ...,Australia Tries to Halt Unrest at Immigration ...
3142,2012-09-10,0,Caribbean coral reefs face collapse. Only 8% ...,Man wakes up from coma after 7 years after giv...,Declassified documents add to proof that US he...,A top story today on Australia's public broadc...,"French newspaper to France's richest man: ""Get...","It Keeps Growing: 41,000 South African Miners...",China's next president injured. Rumored to be...,Hong Kong citizens to China: We refuse your pr...,...,Devastated Mexican father asks: Why did my son...,SA miners 'killed while trying to surrender'. ...,France's richest man denies tax exile bid - LV...,Indian family launch court action for return o...,Iraq vice-president sentenced to death amid de...,"Study finds that in total s, it was the educat...",Hundreds of Palestinians attack Jewish village...,Can Israel Surprise Iran? Maybe Not: Years of ...,Former German first lady battles red-light rum...,"Yet Again, Congo Faces The Specter Of Civil Wa..."
756,2003-03-04,0,How Oakley turns himself on,Owen calls for new sense of adventure,Ridsdale dismissed as 'dimwit' by fans,Premier League set to scrap loan ban,Only Europe has power to lift Ferguson's fadin...,Car

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4101 entries, 0 to 4100
Data columns (total 27 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    4101 non-null   object
 1   Label   4101 non-null   int64 
 2   Top1    4101 non-null   object
 3   Top2    4101 non-null   object
 4   Top3    4101 non-null   object
 5   Top4    4101 non-null   object
 6   Top5    4101 non-null   object
 7   Top6    4101 non-null   object
 8   Top7    4101 non-null   object
 9   Top8    4101 non-null   object
 10  Top9    4101 non-null   object
 11  Top10   4101 non-null   object
 12  Top11   4101 non-null   object
 13  Top12   4101 non-null   object
 14  Top13   4101 non-null   object
 15  Top14   4101 non-null   object
 16  Top15   4101 non-null   object
 17  Top16   4101 non-null   object
 18  Top17   4101 non-null   object
 19  Top18   4101 non-null   object
 20  Top19   4101 non-null   object
 21  Top20   4101 non-null   object
 22  Top21   4101 non-null   

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

# Combine all topic columns into a single text column (try news one by one and see if we can label with 0/1)
df['text'] = df[['Top' + str(i) for i in range(1, 26, 3)]].fillna('').agg(' '.join, axis=1)

# Drop the individual topic columns and keep only 'Date', 'Label', and 'text'
df = df[['Date', 'Label', 'text']]

# Preprocess text data
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(preprocess_text)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['Label'], test_size=0.2, random_state=42)

# Create a pipeline with TF-IDF and RandomForestClassifier
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=2000, ngram_range=(1, 2))),
    ('clf', LogisticRegression(random_state=42, max_iter=200, class_weight='balanced'))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

[nltk_data] Downloading package stopwords to /home/oem/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/oem/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


              precision    recall  f1-score   support

           0       0.47      0.52      0.49       372
           1       0.56      0.51      0.53       449

    accuracy                           0.51       821
   macro avg       0.51      0.51      0.51       821
weighted avg       0.52      0.51      0.52       821



In [6]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Initialize VADER sentiment intensity analyzer
analyzer = SentimentIntensityAnalyzer()

# Apply VADER to each text entry to get compound scores
def get_vader_sentiment(text):
    score = analyzer.polarity_scores(text)['compound']
    # Define thresholds to classify as positive, neutral, or negative based on your Label encoding
    return 1 if score >= 0.01 else 0

df = data.copy()
# Assuming 'text' column has the combined topic text and 'Label' column is your target
df['text'] = df[['Top' + str(i) for i in range(1, 26, 6)]].fillna('').agg(' '.join, axis=1)

# Preprocess text data
# nltk.download('stopwords')
# nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # Remove all non-word characters
    text = text.lower()  # Convert to lowercase
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

df['text'] = df['text'].apply(preprocess_text)

df['vader_sentiment'] = df['text'].apply(get_vader_sentiment)

# Split data
X_train, X_test, y_train, y_test = train_test_split(df['vader_sentiment'], data['Label'], stratify=data['Label'] test_size=0.2, random_state=42)

# Evaluate performance
print(classification_report(y_test, X_test))


SyntaxError: invalid syntax. Perhaps you forgot a comma? (3282371301.py, line 37)

In [ ]:
df.sample(5)